In [1]:
import shutil, os, subprocess
from google.colab import drive
drive.mount('/content/drive')
os.makedirs('/content/drive/MyDrive/llama3b/', exist_ok=True)
os.chdir('/content/drive/MyDrive/llama3b/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
! python --version

Python 3.10.12
/bin/bash: line 1: pytorch: command not found


In [2]:
!pip3 install -U xformers --index-url https://download.pytorch.org/whl/cu121
!pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"

Looking in indexes: https://download.pytorch.org/whl/cu121
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-576cwhjs/unsloth_cfac318fb0cc4db693db12d757d0b3e4
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-576cwhjs/unsloth_cfac318fb0cc4db693db12d757d0b3e4
  Resolved https://github.com/unslothai/unsloth.git to commit 38663b01f5dd0e610b12475bd95b144303cff539
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-vgerpc2d/unsloth_e04df08949ff47bb8d4b2f952b5b1e5f
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-vgerpc2d/unsloth_e04df08949ff47bb8d4b2f952b5b1e5f
  Resolved https://github.com/unslothai/unsloth.git to commit 38663b01f5dd0e610b12475bd95b144303cff539
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [4]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

In [6]:
!pip3 install -U xformers --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 101.7 MB/s eta 0:00:00
  Attempting uninstall: xformers
    Found existing installation: xformers 0.0.25.post1
    Uninstalling xformers-0.0.25.post1:
      Successfully uninstalled xformers-0.0.25.post1


In [5]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
] # More models at https://huggingface.co/unsloth
#model_path = "/content/drive/MyDrive/llama302b/model_mna/" #originally I used my fine-tuned model. The difference between it and an original
#                                                         llama isn't that big - 3-4%. The labeled dataset and the topic in general is full of
#                                                         highly toxic posts. The temperature was 0.2 and top_p=0.3
model_path = "unsloth/llama-3-8b-bnb-4bit"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_path,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.10.0: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
from transformers import GenerationConfig

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# Create a GenerationConfig instance with the desired settings
generation_config = GenerationConfig(
    bos_token_id=128000,
    eos_token_id=128001,
    max_length=4096,
    temperature=0.9,
    top_p=0.5
)

# Apply the generation configuration to the model
model.generation_config = generation_config

# Set pad_token_id
model.generation_config.pad_token_id = tokenizer.pad_token_id

In [7]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

In [8]:
def llama3_talk(instruction, input):

    prompt = alpaca_prompt.format(instruction, input, "")

    inputs = tokenizer([prompt], return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 90)

    return tokenizer.batch_decode(outputs,  skip_special_tokens=True)[0]

#Examples

In [9]:
mna_news_instruction = "Define if the post toxic or non-toxic. If the post is toxic then return 0. If the post is non-toxic then return 1."  # Define the instruction
mna_news_input = "trump is a good and caring person"
inputs = tokenizer([alpaca_prompt.format(mna_news_instruction, mna_news_input, "")], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 90)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])
#print(llama3_talk(mna_news_input, mna_news_instruction))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Define if the post toxic or non-toxic. If the post is toxic then return 0. If the post is non-toxic then return 1.

### Input:
trump is a good and caring person

### Response:
0



In [ ]:
#inputs = tokenizer([alpaca_prompt.format(mna_news_instruction, mna_news_input, "")], return_tensors = "pt").to("cuda")#
#
#outputs = model.generate(**inputs, max_new_tokens = 90)
#
#tokenizer.batch_decode(outputs)[0]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWhich company is the Acquirer. \n\n### Input:\nHCL Technologies has completed acquisition of 51% stake in German IT Consulting Company, Gesellschaft für Banksysteme GmbH (“GBS”). The acquisition was completed on January 05, 2022.\n\n### Response:\nHCL Technologies has completed acquisition of 51% stake in German IT Consulting Company, Gesellschaft für Banksysteme GmbH (“GBS”). The acquisition was completed on January 05, 2022.<|end_of_text|>'

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
#inputs = tokenizer(
#[
#    alpaca_prompt.format(
#        "Continue the fibonnaci sequence.", # instruction
#        "1, 1, 2, 3, 5, 8", # input
#        "", # output - leave this blank for generation!
#    )
#], return_tensors = "pt").to("cuda")
#
#from transformers import TextStreamer
#text_streamer = TextStreamer(tokenizer)
#_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Continue the fibonnaci sequence.

### Input:
1, 1, 2, 3, 5, 8

### Response:
The next number in the sequence is 13.<|end_of_text|>


In [10]:
import json

file_name = "data.json"
with open(file_name, 'r') as f:
    posts_data = json.load(f)

toxicity_map = dict()

In [11]:
mna_instruction = "Define if the post toxic or non-toxic. If the post is toxic then return 0. If the post is non-toxic then return 1."
#first_5_post_ids = list(posts_data.keys())[:100]
#toxicity_map_test = {}

#for post_id in first_5_post_ids:
for post_id, post_data in posts_data.items():
    post_data = posts_data[post_id]
    output = llama3_talk(post_data["content"], mna_instruction)

    if "### Response:" in output:
        answer = output.split("### Response:")[1].strip()
    else:
        answer = output.strip()

    try:
        toxicity_level = int(answer)
    except ValueError:
        toxicity_level = 0

    toxicity_map[post_id] = toxicity_level

KeyboardInterrupt: 

In [24]:
k = list(toxicity_map.values())
d = {0:0, 1:0}
for i, j in toxicity_map.items():
  d[j] += 1

print("The distribution of posts according to toxicity:" , d)
print("Share of toxic posts:", d[0]/932)

The distribution of posts according to toxicity: {0: 932, 1: 0}
Share of toxic posts: 1.0
